# Battery Model
## Problem 
- Build an optimisation model to charge/discharge the battery over the time period provided 
(2018-2020) in order to maximise profits. You may assume that the battery is a price-taker (ie. the 
actions of the battery have no impact on the market prices).
- You can trade in the 3 markets. 
- The battery can export any amount of power up to its maximum discharge rate for any 
duration of time, as long as it has sufficient energy stored to do so (And same with the charging -- needs storage)
- Markets 1 and 2 are traded at half-hourly time granularity, whereas Market 3 is traded at daily 
granularity 
- The battery cannot sell the same unit of power into multiple markets, but can divide its power 
across the markets (The energy you give or take to the markets must add up to the total energy difference)
- For the battery to participate in Markets 1 and 2, it must export/import a constant level of 
power for the full half-hour period 
For the battery to participate in Market 3, it must export/import a constant level of power for 
the full day 
## Inputs
- Parameters are stored in the "config.yml" file
- Price data is stored in csv file - "data/input_data/market12_data.csv" and "data/input_data/market3_data.csv"
- Also there is information about the total production and different types of electricity production
## Outputs
- [ ] Total profits (Revenue - capex - opex*time)
- [ ] Data - Total energy in, Total energy out + any other profits 
- [ ] Plots for all optimisation variables - NICE TO HAVE
- [ ] Short description
- [ ] Dependencies
## Process
- However, I have decided to create 5 tasks 
1. Decide energy in and out from Market 1 with no change in capacity 
2. Decide energy in and out from Market 1 with change in capacity
3. Decide energy in and out from Market 1 and 2 with change in capacity
4. Decide energy in and out from Market 1, 2 and 3 with change in capacity
5. Decide energy in and out from Market 1, 2 and 3 with change in capacity with difference in energy in and out efficiency. **(Explain later)**
6. Decide energy in and out from Market 1, 2 and 3 with change in capacity with weightings for different energy types.

## Variables
- $k$ is the sample number. Here it represents the number of half an hour steps the sample is from the start. 
- The prices $u_1[k]$, $u_2[k]$ and $u_3[k]$ represent the prices of each market at sample $k$. - market 1, 2 and 3 respectively
- The decision variables: 
- The energies into the battery at each sample ${x_{in}}_1[k]$, ${x_{in}}_2[k]$ and ${x_{in}}_3[k]$
- The energies into the battery at each sample ${x_{out}}_1[k]$, ${x_{out}}_2[k]$ and ${x_{out}}_3[k]$
- The net power into the battery at each sample (constant) $P_{in}[k]$
- The net energy of the battery at each sample $E_{net}[k]$
- The number of cycles at each sample $N_{cyc}[k]$
- The maximum capacity of the battery at each sample $C_{max}[k]$
## Model
$$P_{in}[k]=\frac{\Sigma^3_{i=1} {x_{in}}_i[k] - {x_{out}}_i[k]}{\Delta t}$$
$$E_{in}[k+1] = E_{in}[k] +(\Sigma^3_{i=1} {x_{in}}_i[k] - {x_{out}}_i[k])\times\mathtt{charging \ efficiency}$$
$$N_{cyc}[k+1]=N_{cyc}[k]+\frac{|\Sigma^3_{i=1} {x_{in}}_i[k] - {x_{out}}_i[k]|}{(2*C_{max}[k])}$$
- **ASSUMING MAX CAP DOESN'T CHANGE**
$$C_{max}[k+1]=C_{max}[k]$$

## Assumptions
- The battery can discharge any amount of energy up to it's current energy
- The battery can charge any amount of energy up to it's current remaining storage
- The power is constant throughout a sample
- The efficiency of the charging is 95% -- therefore the battery only gets 95% of what it buys
- The efficiency of the discharging is 95% -- therefore the battery only recieves 95% of the money that what the energy discharged cost
- The battery can charge and discharge at the same time
- **The battery maximum capacity does not change with time**
- **The battery can only use market 1**
## Initialisation
$$E_{net}[0]=0\mathtt{\ MW}\\
N_{cyc}[0]=0 \\ 
C_{max}=0\mathtt{\ MW}
$$

## Objective
- We want to maximise the profits by minimise the costs of the charging the batteries 
$$\begin{aligned}
\min_{{x_{in}}_1,{x_{out}}_1,{x_{in}}_2,{x_{out}}_2,{x_{in}}_3,{x_{out}}_3} \Sigma^3_{i=1} ({x_{in}}_i[k] - {x_{out}}_i[k])\times u_i[k]
\end{aligned}$$

## Constraints
- Additional to the model constaints
- Power is limited to the maximum charging and discharging rate 
$$\mathtt{max \ discharging \ rate} <= P_{in}[k] <= \mathtt{max \ discharging \ rate} $$
- The number of cycles is a parameter that must be exceeded
$$ 0 <= N_{cyc}[k] <= \mathtt{max \ cycles}$$
- The number of years is also limited
$$ 0 <= k / 31,536,000 <= \mathtt{max \ cycles}$$
- The energy is limited by its maximum capacity
$$ 0 <= E_{net}[k] <= C_{max}[k]$$
- The maximum capacity, although changes, never goes over the original max capacity
$$ 0 <= C_{max}[k] <= C_{max}[0]$$
## Solver choice 
- Since the problem is linear. I chose the solver - High performance solver for linear optimisation

In [6]:
using IJulia, CSV, DataFrames, JuMP, YAML, Infiltrator, Dates
include("batterymodel.jl")
include("plot_funcs.jl")

input_data = CSV.read("data/input_data/market12_data.csv", DataFrame)
config_data = YAML.load_file("config.yml")

marketprices1 = input_data.Market_1
start_point = Date.(input_data.time[1], "dd/mm/yyyy HH:MM")
params = BatteryModel.BatteryParams(
            config_data["max_charging_rate"],
            config_data["max_discharging_rate"],
            config_data["max_storage_volume"],
            config_data["charging_efficiency"],
            config_data["discharging_efficiency"],
            config_data["lifetime_years"],
            config_data["lifetime_cycles"],
            config_data["degradation_rate"],
            config_data["capex"], 
            config_data["opex"]
)

marketprices1 = marketprices1[1:100]
print("market prices1: $marketprices1")


market prices1: [48.47, 49.81, 53.65, 52.48, 47.25, 47.18, 40.24, 38.56, 37.55, 36.56, 39.25, 41.23, 45.68, 44.83, 48.87, 50.73, 45.01, 48.87, 44.41, 42.59, 46.15, 46.19, 54.83, 60.57, 58.75, 59.88, 58.03, 59.72, 55.94, 52.55, 59.43, 61.01, 63.07, 64.91, 63.27, 61.55, 58.82, 55.67, 46.13, 47.03, 48.91, 45.87, 46.95, 46.71, 46.08, 45.77, 45.25, 39.33, 40.23, 40.6, 38.95, 38.8, 37.28, 37.66, 41.06, 44.07, 42.21, 43.08, 54.59, 55.53, 58.52, 48.16, 43.64, 48.24, 51.96, 53.68, 50.84, 50.46, 51.01, 51.98, 52.38, 52.59, 51.94, 50.2, 46.63, 45.63, 46.09, 45.56, 54.11, 55.46, 53.69, 64.71, 77.94, 73.13, 71.63, 65.18, 60.09, 54.67, 53.43, 49.58, 50.96, 47.73, 47.42, 47.13, 48.52, 43.08, 45.97, 47.84, 51.28, 49.45]

In [7]:
energy_in1, energy_out1, energies, cycle, maximum_capacities = BatteryModel.optimise_battery_charge(marketprices1, params)



Infiltrator.jl needs a fully-functional Julia REPL.
Nearly thereRunning HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [6e-04, 1e+00]
  Cost   [4e+01, 8e+01]
  Bound  [2e+00, 5e+03]
  RHS    [0e+00, 0e+00]
Presolving model
498 rows, 399 cols, 1294 nonzeros  0s
298 rows, 300 cols, 599 nonzeros  0s
1 rows, 2 cols, 2 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-598); columns 0(-600); elements 0(-1695) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00


([-0.0, 0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, -0.0  …  -0.0, 0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0], [0.0, -0.0, 0.0, -0.0, 0.0, -0.0, -0.0, 0.0, -0.0, 0.0  …  0.0, -0.0, 0.0, -0.0, 0.0, -0.0, -0.0, 0.0, -0.0, -0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0  …  -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0], [4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0  …  4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0])

In [8]:
PlotFuncs.plot_battery_performance(marketprices1, energy_in1, energy_out1, energies, cycle, maximum_capacities)

PlotFuncs.write_battery_performance(marketprices1, energy_in1, energy_out1, energies, cycle, maximum_capacities, start_point, params)

LoadError: MethodError: no method matching +(::Date, ::Second)

[0mClosest candidates are:
[0m  +(::TimeType, ::Period, [91m::Period[39m)
[0m[90m   @[39m [35mDates[39m [90m~/.julia/juliaup/julia-1.10.4+0.x64.linux.gnu/share/julia/stdlib/v1.10/Dates/src/[39m[90m[4mperiods.jl:353[24m[39m
[0m  +(::TimeType, ::Period, [91m::Period[39m, [91m::Period...[39m)
[0m[90m   @[39m [35mDates[39m [90m~/.julia/juliaup/julia-1.10.4+0.x64.linux.gnu/share/julia/stdlib/v1.10/Dates/src/[39m[90m[4mperiods.jl:354[24m[39m
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m)
[0m[90m   @[39m [90mBase[39m [90m[4moperators.jl:587[24m[39m
[0m  ...
